In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile



# Will unzip the files so that you can see them..
with zipfile.ZipFile("../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip","r") as z:
    z.extractall(".")

In [ ]:
train_df=pd.read_csv("./train.csv")

test_df=pd.read_csv("./test.csv")

train_df.sample(5)

In [ ]:
cols_target = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']

In [ ]:
train_dfx = pd.read_csv("../input/jigsaw-toxic-comment-support/clean_lemmatized_trained2.csv")
test_dfx = pd.read_csv("../input/jigsaw-toxic-comment-support/clean_lemmatized_test.csv")

In [ ]:
X = train_dfx.comment_text
test_X = test_dfx.comment_text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=5000,stop_words='english')
vect

In [ ]:
X_dtm = vect.fit_transform(X.values.astype('U'))
# examine the document-term matrix created from X_train
X_dtm
# transform the test data using the earlier fitted vocabulary, into a document-term matrix
test_X_dtm = vect.transform(test_X.values.astype('U'))
# examine the document-term matrix from X_test
test_X_dtm

In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
clf = XGBClassifier(use_label_encoder =False,eval_metric='logloss',objective ="binary:logistic")
clf2 = LGBMClassifier()

In [ ]:
# create submission file
submission_binary1 = pd.read_csv('./sample_submission.csv')

for label in cols_target:
    print('... Processing {}'.format(label))
    y = train_df[label]
    # train the model using X_dtm & y
    clf.fit(X_dtm, y)
    # compute the training accuracy
    y_pred_X = clf.predict(X_dtm)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = clf.predict_proba(test_X_dtm)[:,1]
    submission_binary1[label] = test_y_prob

In [ ]:
# create submission file
submission_binary2 = pd.read_csv('./sample_submission.csv')

for label in cols_target:
    print('... Processing {}'.format(label))
    y = train_df[label]
    # train the model using X_dtm & y
    clf2.fit(X_dtm, y)
    # compute the training accuracy
    y_pred_X = clf2.predict(X_dtm)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = clf2.predict_proba(test_X_dtm)[:,1]
    submission_binary2[label] = test_y_prob

In [ ]:
submission_binary1.to_csv('submission_binary_xgb.csv',index=False)
submission_binary2.to_csv('submission_binary_lightgbm.csv',index=False)